# TeAAL specifications on variants of Loops' SpMV implementations (Thread-Mapped)

Description: Assign a fixed, constant number of work tiles to each thread. Resultant work items from each work tile are processed sequentially within the thread. 

In this version, each `tile` represents a row of $A$, and each `atom` represents a nonzero entry of $A$. In other words, assigning a thread to each row of $A$. Therefore, TeAAL specs is idential to the original.

Template: https://github.com/gunrock/loops/blob/main/include/loops/algorithms/spmv/thread_mapped.cuh

Scheduler (referenced as `config` below): https://github.com/gunrock/loops/blob/main/include/loops/schedule/thread_mapped.hxx

GPU Kernel Template (Use it as a reference, don't execute it):

In [ ]:
__global__ void __thread_mapped(...) {
  for (auto row : config.tiles()) { # Loop over assigned rows for each tid. TeAAL: For each thread, loop over partitioned I.
    type_t sum = 0;

    for (auto nz : config.atoms(row)) {  # Loop over nonzero entry of current row. TeAAL: Loop over J of current i-th row. 
      sum += values[nz] * x[indices[nz]];
    }

    // Output
    y[row] = sum;
  }
}

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "../../")

from src import utils

## Initialization

Initialize the input tensors.

For simplicity, the size of a thread warp is the same as the size of a thread block (`WARP_SIZE = BLOCK_SIZE`). Suppose that each GPU SM processes 1 thread warp/block per cycle.

In [ ]:
I = 8
J = 8

# Hardware Specification
NUM_SM = 2 # Number of GPU SMs 
WARP_SIZE = 2 # Number of threads per warp
NUM_THREADS = NUM_SM * WARP_SIZE # Total number of threads

print(f"Hardware Specification\n  NUM_SM: {NUM_SM}, WARP_SIZE: {WARP_SIZE}, NUM_THREADS: {NUM_THREADS}")

seed = 1

A_IJ = Tensor.fromRandom(rank_ids=["I", "J"], shape=[I, J], seed=seed, density=[0.9, 0.6], name="A")
B_J = Tensor.fromRandom(rank_ids=["J"], shape=[J], seed=seed + 1, density=[1], name="B")

## TeAAL Specifications

Rows of matrix $A$ are partitioned across the SMs' threads. A thread can be assigned to a row with all zeros. 

Note that the current TeAAL specificaiton does not allow to specify the rank of `opt: slip`. This means there exists a synchronization across the SMs.

In [ ]:
yaml = """
einsum:
  declaration:
    A: [I, J]
    B: [J]
    Z: [I]
  expressions:
    - Z[i] = A[i, j] * B[j]
mapping:
  rank-order:
    A: [I, J]
    B: [J]
    Z: [I]
  partitioning:
    Z:
      I: [uniform_shape(NUM_THREADS)]
  loop-order:
    Z: [I1, I0, J]
    # I1: Number of partitioned rows (I)
    # I0: Size of each partitioned row = NUM_THREADS
  spacetime:
    Z:
      space: [I0] # Parallelize over NUM_THREADS
      time: [I1, J] 
"""

utils.compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell).

In [ ]:
utils.check_matrix_vector_mul(A_IJ, B_J, Z_I)